In [1]:
import random 

In [191]:
def dice_roll(x = 20, y = 0, modifier = 0, numdice = 1, crit = False):
    ''' dice_roll(x, modifier, numdice)
        
        x = int n sides of the dice (default d20)
        y = int n sides of added die, granted from a spell like Guidance (default 0)
        modifier = int value of modifier for roll (default 0)
        numdice = int n number of dice (default 1)
        crit = whether or not the hit roll was a critical hit (default False)
        

        This function returns the final total for a roll in D&D
        Prints each dice roll, base sum, modifier, and the total roll value
        This function handles critical hits by duplicating the damage die, then adding the ability modifier.
        
        
        Returns dice, additional_dice, modifier
    '''
    count = 1
    dice = 0
    add_dice = 0
    output = ''
    for i in range(numdice):
        count += 1
        roll = random.randint(1,x)
        output += f'<|>  d{x} #{count - 1} Dice Roll :  {roll}  <|>\n'
        dice += roll
    if crit == True:
        dice *= 2
    if y != 0:
            add_dice += random.randint(1,y)
            output += f'<|>  d{y} Bonus Roll   :  {add_dice}  <|>\n'
    output += f'''\n\n Dice Total          :  {dice + add_dice}\n Modifier            :  {modifier:{"+" if modifier else ""}}\n-----------------------------\n Total Roll          :  {dice + modifier + add_dice}'''
    return dice, add_dice, modifier, output

In [192]:
def roll_d20(addl_dice = 0, modifier = 0):
    ''' addl_dice = int n sides of bonus dice
        modifier = int ability modifier

        returns base sum of dice, bonus dice, modifier, and an output 
            string with total. '''
    return dice_roll(20, addl_dice, modifier, 1, False)
    

In [193]:
dice_type = 6
addl_dice = 4
modifier = +3
number_of_dice = 5
hit_roll = False
dice, add_dice, modifier, output = dice_roll(dice_type, 
                                             addl_dice, 
                                             modifier, 
                                             number_of_dice, 
                                             damage_roll) 
print(output)

<|>  d6 #1 Dice Roll :  4  <|>
<|>  d6 #2 Dice Roll :  5  <|>
<|>  d6 #3 Dice Roll :  2  <|>
<|>  d6 #4 Dice Roll :  1  <|>
<|>  d6 #5 Dice Roll :  4  <|>
<|>  d4 Bonus Roll   :  2  <|>


 Dice Total          :  34
 Modifier            :  +3
-----------------------------
 Total Roll          :  37


In [194]:
modifier = +3
addl_dice = 0
hit_roll = True
dice, add_dice, modifier, output = roll_d20(addl_dice,
                                            modifier)
print(output)

<|>  d20 #1 Dice Roll :  17  <|>


 Dice Total          :  17
 Modifier            :  +3
-----------------------------
 Total Roll          :  20
